# Creacion de transformadores y PipeLines Personalizados 

* > En este notebook se muestran la creacion de transformadores y PipeLines Personalizados

# Preparacion del Dataset

En este notebook se muestran algunas de las tecnicas mas utilizadas para transformar el Dataset.

## Dataset

## Descripcion
ISCX NSL-KDD is a data set suggested to solve some of the inherent problems of the KDD'99 data set which are mentioned in [1]. Although, this new version of the KDD data set still suffers from some of the problems discussed by McHugh and may not be a perfect representative of existing real networks, because of the lack of public data sets for network-based IDSs, we believe it still can be applied as an effective benchmark data set to help researchers compare different intrusion detection methods.

Furthermore, the number of records in the NSL-KDD train and test sets are reasonable. This advantage makes it affordable to run the experiments on the complete set without the need to randomly select a small portion. Consequently, evaluation results of different research work will be consistent and comparable.

Data files

* <spam style="color:green">  KDDTrain+.ARFF: The full NSL-KDD train set with binary labels in ARFF format </spam>
* KDDTrain+.TXT: The full NSL-KDD train set including attack-type labels and difficulty level in CSV format
* KDDTrain+_20Percent.ARFF: A 20% subset of the KDDTrain+.arff file
* KDDTrain+_20Percent.TXT: A 20% subset of the KDDTrain+.txt file
* KDDTest+.ARFF: The full NSL-KDD test set with binary labels in ARFF format
* KDDTest+.TXT: The full NSL-KDD test set including attack-type labels and difficulty level in CSV format
* KDDTest-21.ARFF: A subset of the KDDTest+.arff file which does not include records with difficulty level of 21 out of 21
* KDDTest-21.TXT: A subset of the KDDTest+.txt file which does not include records with difficulty level of 21 out of 21

[Link de Descarga](https://www.unb.ca/cic/datasets/nsl.html) 

License
You may redistribute, republish, and mirror the ISCX NSL-KDD dataset in any form. However, any use or redistribution of the data must include a citation to the NSL-KDD dataset and the paper referenced below.

References: [1] M. Tavallaee, E. Bagheri, W. Lu, and A. Ghorbani, “A Detailed Analysis of the KDD CUP 99 Data Set,” Submitted to Second IEEE Symposium on Computational Intelligence for Security and Defense Applications (CISDA), 2009.

### Imports


In [1]:
import arff 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline

## Funsiones Auxiliares 

In [2]:
def load_kdd_dataset(data_path): 
    """Lectura del dataset NSL-KDD"""
    with open(data_path, 'r') as train_set:
        dataset = arff.load(train_set)
        attributes = [attr[0] for attr in dataset["attributes"]]
    return pd.DataFrame(dataset["data"], columns=attributes)

In [3]:
def train_val_test_split(df, rstate=42, shuffle=True, stratify=None):
    print("Longitud del DataSet:", len(df))
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
        df, test_size=0.4, random_state=rstate, shuffle=shuffle, stratify=strat)
    strat = test_set[stratify] if stratify else None
    val_set, test_set = train_test_split(
        test_set, test_size=0.5, random_state=rstate, shuffle=shuffle, stratify=strat)
    return (train_set, val_set, test_set)

### Lectura del DataSet

In [4]:
df = load_kdd_dataset("./datasets/datasets/NSL-KDD/KDDTrain+.arff")

### 2.- Division del DataSet


In [5]:
train_set, val_set, test_set = train_val_test_split(df, stratify='protocol_type')

Longitud del DataSet: 125973


In [6]:
print("Longitud del training set:", len(train_set))
print("Longitud del validation set:", len(val_set)) # Corrección de sintaxis y variable
print("Longitud del test set:", len(test_set))

Longitud del training set: 75583
Longitud del validation set: 25195
Longitud del test set: 25195


## API Sklearn

Antes de continuar es nesesario hacer una paquena resena sobre como funsionan las Api's de Sklearn
* **Estimators**: Cualquier objeto que pueda estimar algun parametro: 
    * El propio estimador se forma mediante el metodo fit(), que siempre toma un DataSet como argumento.
    * Cualquier otro parametro de este metodo, es un hiperparametro
* **Transformers**: Son estimadores capaces de transformar el conjunto de datos (como Imputer).
    * La transformacion se realiza mediante el metodo transform()
    * Reciben un DataSet como parametro de entrada.

* **Predictors**: Son estimadores capaces de realizar predicciones.
    * La prediccion se realiza mediante el metodo predict()
    * Reciben un DataSet como entrada.
    * Retornan un DataSet como entrada.
    * Tienen un metodo score() para evaluar el resultado de la prediccion

## 1.- Construyendo transformadores personalizados 

La union de los transformadores propios permite mantener el codigo mucho mas limpio y estructurado ala ahora de preparar los datos por los algoritmos de ML. Ademas, facilitan la reutilizacion de codigo para otros proyectos.


Antes de comenzar,m es necesario recuperar el DataSet limpio y separar las etiquetas del resto de los datos, no necesariamente se quiere aplicar las mismas transformcaciones en ambos conjuntos

In [7]:
# division del DataSet
X_train = train_set.drop('class', axis=1)
y_train = train_set['class'].copy()

In [8]:
# Para ilustar estas seccion es necesario anadir algunos valores nulos a algunas caraaracteristicas del DataSet

X_train.loc[(X_train["src_bytes"]>400)& (X_train["src_bytes"]<800), "src_bytes"] = np.nan 
X_train.loc[(X_train["dst_bytes"]>500)&(X_train["dst_bytes"]<200), "dst_bytes"] =np.nan
X_train

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
113467,0.0,tcp,http,SF,NaN,53508.0,0,0.0,0.0,0.0,...,9.0,255.0,1.00,0.00,0.11,0.03,0.00,0.00,0.0,0.0
31899,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.00,1.00,0.0,0.0
108116,0.0,tcp,http,SF,304.0,636.0,0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.00,0.00,0.0,0.0
89913,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.00,1.00,0.0,0.0
106319,0.0,icmp,eco_i,SF,8.0,0.0,0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64559,0.0,tcp,systat,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,20.0,0.08,0.06,0.00,0.00,1.00,1.00,0.0,0.0
67272,0.0,tcp,http,SF,210.0,736.0,0,0.0,0.0,0.0,...,119.0,255.0,1.00,0.00,0.01,0.02,0.02,0.01,0.0,0.0
32452,3.0,tcp,smtp,SF,889.0,328.0,0,0.0,0.0,0.0,...,111.0,155.0,0.64,0.04,0.01,0.01,0.01,0.00,0.0,0.0
112657,0.0,tcp,http,SF,284.0,444.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0


# Transformadores para atributos numericos

In [9]:

from sklearn.base import BaseEstimator, TransformerMixin

class DeleteNanRows(BaseEstimator, TransformerMixin):
	"""Transformador personalizado para eliminar filas con valores NaN"""
	def __init__(self):
		pass
	def fit(self, X, y=None):
		return self
	def transform(self, X):
		return X.dropna()

delete_nan = DeleteNanRows()

X_train_clean = delete_nan.fit_transform(X_train)
X_train_clean

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
31899,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.00,1.00,0.0,0.0
108116,0.0,tcp,http,SF,304.0,636.0,0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.00,0.00,0.0,0.0
89913,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.00,1.00,0.0,0.0
106319,0.0,icmp,eco_i,SF,8.0,0.0,0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.00,0.00,0.0,0.0
98007,0.0,udp,domain_u,SF,46.0,139.0,0,0.0,0.0,0.0,...,255.0,254.0,1.00,0.01,0.00,0.00,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64559,0.0,tcp,systat,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,20.0,0.08,0.06,0.00,0.00,1.00,1.00,0.0,0.0
67272,0.0,tcp,http,SF,210.0,736.0,0,0.0,0.0,0.0,...,119.0,255.0,1.00,0.00,0.01,0.02,0.02,0.01,0.0,0.0
32452,3.0,tcp,smtp,SF,889.0,328.0,0,0.0,0.0,0.0,...,111.0,155.0,0.64,0.04,0.01,0.01,0.01,0.00,0.0,0.0
112657,0.0,tcp,http,SF,284.0,444.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0


In [10]:
# Transformador disenado para escalar de manera sensilla unicamente unas columnas seleccionadas 

class CustomScaler(BaseEstimator, TransformerMixin):

    def __init__(self, atributes):
        self.atributes = atributes

    def fit(self, X, y=None):
        
        self.scaler_ = RobustScaler()
        self.scaler_.fit(X[self.atributes])
        return self

    def transform(self, X, y=None):
        
        X_copy = X.copy()
        X_copy[self.atributes] = self.scaler_.transform(X_copy[self.atributes])
        return X_copy

In [11]:
custom_scaler = CustomScaler(atributes=['src_bytes', 'dst_bytes'])
X_train_train = custom_scaler.fit_transform(X_train_clean)
X_train_train.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
31899,0.0,tcp,private,S0,-0.170635,0.000000,0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.0,1.0,0.0,0.0
108116,0.0,tcp,http,SF,1.035714,1.133690,0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.0,0.0,0.0,0.0
89913,0.0,tcp,private,S0,-0.170635,0.000000,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.0,1.0,0.0,0.0
106319,0.0,icmp,eco_i,SF,-0.138889,0.000000,0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.0,0.0,0.0,0.0
98007,0.0,udp,domain_u,SF,0.011905,0.247772,0,0.0,0.0,0.0,...,255.0,254.0,1.00,0.01,0.00,0.00,0.0,0.0,0.0,0.0
16447,0.0,tcp,smtp,SF,6.932540,0.647059,0,0.0,0.0,0.0,...,141.0,137.0,0.55,0.04,0.01,0.01,0.0,0.0,0.0,0.0
100052,0.0,tcp,http,SF,0.646825,2.659537,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
28800,0.0,tcp,ftp_data,SF,1.154762,0.000000,0,0.0,0.0,0.0,...,8.0,28.0,1.00,0.00,1.00,0.11,0.0,0.0,0.0,0.0
78082,0.0,tcp,ftp_data,SF,41.055556,0.000000,0,0.0,0.0,0.0,...,93.0,51.0,0.23,0.03,0.23,0.04,0.0,0.0,0.0,0.0
69315,0.0,tcp,systat,S0,-0.170635,0.000000,0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.0,0.0


In [12]:
delete_nan = DeleteNanRows()
X_train_prep = delete_nan.fit_transform(X_train)

In [13]:
X_train_prep.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
31899,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.0,1.0,0.0,0.0
108116,0.0,tcp,http,SF,304.0,636.0,0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.0,0.0,0.0,0.0
89913,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.0,1.0,0.0,0.0
106319,0.0,icmp,eco_i,SF,8.0,0.0,0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.0,0.0,0.0,0.0
98007,0.0,udp,domain_u,SF,46.0,139.0,0,0.0,0.0,0.0,...,255.0,254.0,1.00,0.01,0.00,0.00,0.0,0.0,0.0,0.0
16447,0.0,tcp,smtp,SF,1790.0,363.0,0,0.0,0.0,0.0,...,141.0,137.0,0.55,0.04,0.01,0.01,0.0,0.0,0.0,0.0
100052,0.0,tcp,http,SF,206.0,1492.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0
28800,0.0,tcp,ftp_data,SF,334.0,0.0,0,0.0,0.0,0.0,...,8.0,28.0,1.00,0.00,1.00,0.11,0.0,0.0,0.0,0.0
78082,0.0,tcp,ftp_data,SF,10389.0,0.0,0,0.0,0.0,0.0,...,93.0,51.0,0.23,0.03,0.23,0.04,0.0,0.0,0.0,0.0
69315,0.0,tcp,systat,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,5.0,0.02,0.07,0.00,0.00,1.0,1.0,0.0,0.0


In [14]:
# transformador paa codificar unicamente las acolumnas y devolver un DataFrame

class CustomOneHotEncoder(BaseEstimator, TransformerMixin):

    def __init__ (self):
        self.oh = CustomOneHotEncoder(sparce=False) 
    def fit(self, X, y=None):
        X_cat = X.select_dtypes(include=['object'])
        self.colums_ = pd.get_dummies(X_cat).columns
        self.oh.fit(X_cat)
        return self
    def transform(self, X, y=None):
        X_copy = X.copy()
        X_cat = X_copy.select_dtypes(include=['object'])
        X_num = X_copy.select_dtypes(exclude=['object'])
        X_cat_oh = self.oh.transform(X_cat)
        X_cat_oh_df = pd.DataFrame(X_cat_oh, columns=self.colums_, index=X.index)
        X_copy.drop(columns=X_cat.columns, inplace=True)
        
        return X_copy.join(X_cat_oh_df)

## 6.- Construyendo un Pipeline personalizado

Los Pipelines nos permiten agrupar en un flujo de ejecucion todas las operaciones de transformacion 
que se necesitan realizar sobre un conjunto de datos, esto facilita mucho las transformaciones para diferente conjuntos de datos, los puntos a tener en cuenta son:
* * Recibe un conjunto de datos pares (Nombre, estimador) 
* * Todos menos el ultimo deben ser transformadores.
* * El Pipeline expone los mismos metodos que el **ultimo transfomador**.
* * Cuando se llama al metodo fit del Pipeline, se llama secuencialmente al metodo fit_transform de los transformadores y se les pasa de manera secuencial el output del aterior como input del siguiente.El ultimo invoca al metodo fit

In [15]:
# Construccion de un Pipeline para los atributos numericos
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer


num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('rbst_scaler', RobustScaler())
])



In [16]:
# la clase imputer no admite valores categoricos, se eliminan los atributos categoricos

X_train_num = X_train.select_dtypes(exclude=['object'])

X_train_prep = num_pipeline.fit_transform(X_train_num)

X_train_prep = pd.DataFrame(X_train_prep, columns = X_train_num.columns, index=X_train_num.index)

In [17]:
X_train_prep.head(10)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
113467,0.0,0.000000,101.920000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.421965,0.787755,0.515789,-0.428571,1.833333,1.5,0.00,0.00,0.0,0.0
31899,0.0,-0.174089,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.236735,-0.515789,0.285714,0.000000,0.0,1.00,1.00,0.0,0.0
108116,0.0,1.056680,1.211429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.248555,0.787755,0.515789,-0.428571,0.500000,3.0,0.00,0.00,0.0,0.0
89913,0.0,-0.174089,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.191837,-0.473684,0.571429,0.000000,0.0,1.00,1.00,0.0,0.0
106319,0.0,-0.141700,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.462428,-0.224490,0.515789,-0.428571,16.666667,28.5,0.00,0.00,0.0,0.0
98007,0.0,0.012146,0.264762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.783673,0.515789,-0.285714,0.000000,0.0,0.00,0.00,0.0,0.0
16447,0.0,7.072874,0.691429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.658960,0.306122,0.042105,0.142857,0.166667,0.5,0.00,0.00,0.0,0.0
64957,1.0,0.000000,0.626667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.329480,0.485714,0.147368,0.000000,0.166667,0.5,0.02,0.02,0.0,0.0
100052,0.0,0.659919,2.841905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.787755,0.515789,-0.428571,0.000000,0.0,0.00,0.00,0.0,0.0
28800,0.0,1.178138,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.427746,-0.138776,0.515789,-0.428571,16.666667,5.5,0.00,0.00,0.0,0.0


In [18]:
X_train_prep.head(10)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
113467,0.0,0.000000,101.920000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.421965,0.787755,0.515789,-0.428571,1.833333,1.5,0.00,0.00,0.0,0.0
31899,0.0,-0.174089,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.236735,-0.515789,0.285714,0.000000,0.0,1.00,1.00,0.0,0.0
108116,0.0,1.056680,1.211429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.248555,0.787755,0.515789,-0.428571,0.500000,3.0,0.00,0.00,0.0,0.0
89913,0.0,-0.174089,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,-0.191837,-0.473684,0.571429,0.000000,0.0,1.00,1.00,0.0,0.0
106319,0.0,-0.141700,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.462428,-0.224490,0.515789,-0.428571,16.666667,28.5,0.00,0.00,0.0,0.0
98007,0.0,0.012146,0.264762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.783673,0.515789,-0.285714,0.000000,0.0,0.00,0.00,0.0,0.0
16447,0.0,7.072874,0.691429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.658960,0.306122,0.042105,0.142857,0.166667,0.5,0.00,0.00,0.0,0.0
64957,1.0,0.000000,0.626667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.329480,0.485714,0.147368,0.000000,0.166667,0.5,0.02,0.02,0.0,0.0
100052,0.0,0.659919,2.841905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.787755,0.515789,-0.428571,0.000000,0.0,0.00,0.00,0.0,0.0
28800,0.0,1.178138,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.427746,-0.138776,0.515789,-0.428571,16.666667,5.5,0.00,0.00,0.0,0.0


A continuacion se presenta el metodo column_transformer quu ejecuta todos los pipelines en paralelo y une los resultados para ello e; resultado de los pipelines en paralelo y concatena el resultado, para ello de los pipelines debe ser un valor numerico.

In [21]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_atributes = list(X_train.select_dtypes(exclude=['object']).columns)
cat_atributes = list(X_train.select_dtypes(include=['object']).columns)

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_atributes),
    ('cat', OneHotEncoder(), cat_atributes)
])


X_train_prep = full_pipeline.fit_transform(X_train)
X_train_prep = pd.DataFrame(X_train_prep, columns = list(pd.get_dummies(X_train)), index= X_train.index)

X_train_prep.head(10)

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,flag_S3,flag_SF,flag_SH,land_0,land_1,logged_in_0,logged_in_1,is_host_login_0,is_guest_login_0,is_guest_login_1
113467,0.0,0.000000,101.920000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
31899,0.0,-0.174089,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
108116,0.0,1.056680,1.211429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
89913,0.0,-0.174089,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
106319,0.0,-0.141700,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
98007,0.0,0.012146,0.264762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0
16447,0.0,7.072874,0.691429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
64957,1.0,0.000000,0.626667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
100052,0.0,0.659919,2.841905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
28800,0.0,1.178138,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0


In [20]:
X_train.head(10)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
113467,0.0,tcp,http,SF,NaN,53508.0,0,0.0,0.0,0.0,...,9.0,255.0,1.00,0.00,0.11,0.03,0.00,0.00,0.0,0.0
31899,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,4.0,0.02,0.05,0.00,0.00,1.00,1.00,0.0,0.0
108116,0.0,tcp,http,SF,304.0,636.0,0,0.0,0.0,0.0,...,39.0,255.0,1.00,0.00,0.03,0.06,0.00,0.00,0.0,0.0
89913,0.0,tcp,private,S0,0.0,0.0,0,0.0,0.0,0.0,...,255.0,15.0,0.06,0.07,0.00,0.00,1.00,1.00,0.0,0.0
106319,0.0,icmp,eco_i,SF,8.0,0.0,0,0.0,0.0,0.0,...,2.0,7.0,1.00,0.00,1.00,0.57,0.00,0.00,0.0,0.0
98007,0.0,udp,domain_u,SF,46.0,139.0,0,0.0,0.0,0.0,...,255.0,254.0,1.00,0.01,0.00,0.00,0.00,0.00,0.0,0.0
16447,0.0,tcp,smtp,SF,1790.0,363.0,0,0.0,0.0,0.0,...,141.0,137.0,0.55,0.04,0.01,0.01,0.00,0.00,0.0,0.0
64957,1.0,tcp,smtp,SF,NaN,329.0,0,0.0,0.0,0.0,...,198.0,181.0,0.65,0.03,0.01,0.01,0.02,0.02,0.0,0.0
100052,0.0,tcp,http,SF,206.0,1492.0,0,0.0,0.0,0.0,...,255.0,255.0,1.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0
28800,0.0,tcp,ftp_data,SF,334.0,0.0,0,0.0,0.0,0.0,...,8.0,28.0,1.00,0.00,1.00,0.11,0.00,0.00,0.0,0.0
